In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import shutil
import sys
import time
import pprint
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl, np, pd, sklearn, tf, keras:
#     print(module.__name__, module.__version__)
housing = fetch_california_housing()
# print(housing  .DESCR)
# print(housing.data.shape)
# print(housing.target.shape)
# pprint.pprint(housing.data[0:5])
# pprint.pprint(housing.target[0:5])
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)
# print(x_train.shape, y_train.shape)
# print(x_valid.shape, y_valid.shape)
# print(x_test.shape, y_test.shape)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [6]:
#RandomizedSearchCV
#1.转化为sklearn的model
#2.定义参数集合
#3.搜索参数

def build_model(hidden_layers = 1, layer_size=30,learning_rate=3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(
        layer_size,activation='relu',input_shape=x_train.shape[1:]))

    for _ in range(hidden_layers - 1):
        model.add(keras.layers.Dense(layer_size))
        model.add(keras.layers.BatchNormalization())  #批归一化
        model.add(keras.layers.Activation('relu'))
    
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss='mse',optimizer=optimizer)
    return model

callbacks = [keras.callbacks.EarlyStopping(
    patience=10, min_delta=1e-2)] 
    #连续5次损失函数和上一次差距小于1e-3即可停止训练    
sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [13]:
from scipy.stats import reciprocal
#f(x) = 1/(x*log(b/a)) a<= x <=b

param_distribution={
    "hidden_layers":[1,2,3,4],
    "layer_size":np.arange(1,100),
    "learning_rate":reciprocal(1e-4,1e-2)
}

from sklearn.model_selection import RandomizedSearchCV

random_search_CV = RandomizedSearchCV(sklearn_model,
                                      param_distribution,
                                      n_iter = 10,
                                      n_jobs = 1)

random_search_CV.fit(x_train_scaled,y_train,epochs=100,
                     validation_data=(x_valid_scaled,y_valid),
                     callbacks=callbacks)
#cross validation 训练集分成n份，n-1份用于训练，1份用于验证

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 145us/sample - loss: 2.0379 - val_loss: 2.0067
Epoch 2/100
7740/7740 [==============================] - 0s 55us/sample - loss: 0.7396 - val_loss: 0.7596
Epoch 3/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.6461 - val_loss: 0.6268
Epoch 4/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5911 - val_loss: 0.5577
Epoch 5/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5488 - val_loss: 0.5649
Epoch 6/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5332 - val_loss: 0.5091
Epoch 7/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5149 - val_loss: 0.4888
Epoch 8/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.4942 - val_loss: 0.4937
Epoch 9/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.4784 - val_loss: 0.47

7740/7740 [==============================] - 0s 52us/sample - loss: 0.7032 - val_loss: 0.7941
Epoch 3/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6239 - val_loss: 0.6358
Epoch 4/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.5800 - val_loss: 0.6022
Epoch 5/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.5471 - val_loss: 0.6248
Epoch 6/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.5282 - val_loss: 0.5520
Epoch 7/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.5193 - val_loss: 0.5166
Epoch 8/100
7740/7740 [==============================] - 0s 55us/sample - loss: 0.5092 - val_loss: 0.5408
Epoch 9/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.4821 - val_loss: 0.5005
Epoch 10/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.4863 - val_loss: 0.5093
Epoch 11/100
7740/7740 [==============================] -

Epoch 30/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4404 - val_loss: 0.4939
Epoch 31/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.4314 - val_loss: 0.4823
Epoch 32/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.4414 - val_loss: 0.4834
Epoch 33/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.4246 - val_loss: 0.4758
Epoch 34/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4315 - val_loss: 0.4632
Epoch 35/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4250 - val_loss: 0.4690
Epoch 36/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4162 - val_loss: 0.4653
Epoch 37/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4374 - val_loss: 0.4605
Epoch 38/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4157 - val_loss: 0.4556
Epoch 39/100
7740/7740 [=============

Epoch 37/100
7740/7740 [==============================] - 1s 65us/sample - loss: 0.4321 - val_loss: 0.4533
Epoch 38/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.4247 - val_loss: 0.5107
Epoch 39/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.4343 - val_loss: 0.5240
Epoch 40/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4168 - val_loss: 0.4680
Epoch 41/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4170 - val_loss: 0.4501
Epoch 42/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4190 - val_loss: 0.4517
Epoch 43/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4163 - val_loss: 0.5129
Epoch 44/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.4192 - val_loss: 0.5247
Epoch 45/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.4208 - val_loss: 0.4607
Epoch 46/100
7740/7740 [=============

7740/7740 [==============================] - 0s 37us/sample - loss: 1.0107 - val_loss: 1.0286
Epoch 10/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.9523 - val_loss: 0.9778
Epoch 11/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.9078 - val_loss: 0.9384
Epoch 12/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.8731 - val_loss: 0.9071
Epoch 13/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.8449 - val_loss: 0.8815
Epoch 14/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.8217 - val_loss: 0.8602
Epoch 15/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.8021 - val_loss: 0.8420
Epoch 16/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.7852 - val_loss: 0.8262
Epoch 17/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.7706 - val_loss: 0.8125
Epoch 18/100
7740/7740 [==========================

...

Epoch 28/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6366 - val_loss: 0.7115
Epoch 29/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6316 - val_loss: 0.7054
Epoch 30/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6271 - val_loss: 0.6999
Epoch 31/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6230 - val_loss: 0.6949
Epoch 32/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6192 - val_loss: 0.6903
Epoch 33/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6156 - val_loss: 0.6861
Epoch 34/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6123 - val_loss: 0.6821
Epoch 35/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6092 - val_loss: 0.6784
Epoch 36/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6062 - val_loss: 0.6749
Epoch 37/100
7740/7740 [=============

Epoch 79/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5589 - val_loss: 0.5837
Epoch 80/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5573 - val_loss: 0.5820
Epoch 81/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5557 - val_loss: 0.5803
Epoch 82/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5541 - val_loss: 0.5786
Epoch 83/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5525 - val_loss: 0.5769
Epoch 84/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5510 - val_loss: 0.5753
Epoch 85/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5495 - val_loss: 0.5737
Epoch 86/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5480 - val_loss: 0.5722
Epoch 87/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5465 - val_loss: 0.5705
Epoch 88/100
7740/7740 [=============

Epoch 54/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5486 - val_loss: 0.5946
Epoch 55/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5471 - val_loss: 0.5930
Epoch 56/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5457 - val_loss: 0.5916
Epoch 57/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5443 - val_loss: 0.5901
Epoch 58/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5430 - val_loss: 0.5887
Epoch 59/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5417 - val_loss: 0.5872
Epoch 60/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5404 - val_loss: 0.5859
Epoch 61/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5391 - val_loss: 0.5846
Epoch 62/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.5379 - val_loss: 0.5832
Epoch 63/100
7740/7740 [=============

Epoch 43/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6167 - val_loss: 0.6831
Epoch 44/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6142 - val_loss: 0.6805
Epoch 45/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.6118 - val_loss: 0.6780
Epoch 46/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6095 - val_loss: 0.6755
Epoch 47/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6072 - val_loss: 0.6732
Epoch 48/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6050 - val_loss: 0.6708
Epoch 49/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6028 - val_loss: 0.6685
Epoch 50/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.6007 - val_loss: 0.6662
Epoch 51/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.5986 - val_loss: 0.6639
Epoch 52/100
7740/7740 [=============

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x147b615c0>,
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'hidden_layers': [1, 2, 3, 4], 'layer_size': array([ 1,  2, ..., 98, 99]), 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x148e27390>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [15]:
print(random_search_CV.best_params_)
print(random_search_CV.best_score_)
print(random_search_CV.best_estimator_)

{'hidden_layers': 1, 'layer_size': 85, 'learning_rate': 0.004043079980561536}
-0.36650148638613156


In [19]:
model_best = random_search_CV.best_estimator_.model
model_best.evaluate(x_test_scaled, y_test)

5160/5160 [==============================] - 0s 16us/sample - loss: 0.3830


0.38295542690180995